# **1. Perkenalan Dataset**

Eksperimen ini menggunakan **Breast Cancer Wisconsin (Diagnostic)** yang berisi 569 sampel tumor payudara dengan 30 fitur numerik hasil ekstraksi citra. Target berisi label 0 (malignant) dan 1 (benign). Tujuan: menyiapkan data bersih, terstandarisasi untuk klasifikasi biner.

Alasan pemilihan dataset:
- Bukan dataset latihan di modul (bukan Iris/MNIST/Wine/Credit).
- Fitur numerik sehingga pipeline preprocessing sederhana.
- Ukuran sedang (569 baris) memudahkan eksperimen cepat.

# **2. Import Library**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)
sns.set(style='whitegrid')

# Lokasi file relatif terhadap notebook
BASE_DIR = Path.cwd()
RAW_PATH = (BASE_DIR / '../namadataset_raw/breast_cancer_raw.csv').resolve()
PREP_DIR = BASE_DIR / 'namadataset_preprocessing'
PREP_DIR.mkdir(exist_ok=True)
RAW_PATH

# **3. Memuat Dataset**

Membaca dataset mentah dari folder 
amadataset_raw lalu menampilkan beberapa baris awal untuk verifikasi.

In [ ]:
df = pd.read_csv(RAW_PATH)
print('Dataset shape:', df.shape)
df.head()

# **4. Exploratory Data Analysis (EDA)**

Pemeriksaan struktur data, nilai kosong, dan duplikasi.

In [ ]:
print('Struktur kolom:')
df.info()

print('
Ringkasan statistik:')
display(df.describe())

missing = df.isna().sum()
print('
Jumlah nilai kosong per kolom:
', missing)

duplicates = df.duplicated().sum()
print('
Jumlah baris duplikat:', duplicates)


Distribusi label untuk memastikan dataset tidak imbang berat sebelah.

In [ ]:
target_counts = df['target'].value_counts().rename({0:'malignant', 1:'benign'})
print(target_counts)

sns.countplot(x=df['target'].map({0:'malignant',1:'benign'}))
plt.title('Distribusi Label Target')
plt.ylabel('Count')
plt.xlabel('Label')
plt.show()


Korelasi fitur (subset 10 fitur pertama) untuk melihat hubungan antar fitur.

In [ ]:
subset_cols = df.columns[:10]
corr = df[subset_cols].corr()
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=False, cmap='Blues')
plt.title('Korelasi 10 fitur pertama')
plt.show()


# **5. Data Preprocessing**

Langkah:
- Drop duplikasi.
- Pisahkan fitur dan target.
- Imputasi median (antisipasi missing).
- Standardisasi fitur numerik.
- Simpan dataset siap latih ke 
amadataset_preprocessing/breast_cancer_preprocessed.csv.

In [ ]:
df_clean = df.drop_duplicates().copy()
feature_cols = [c for c in df_clean.columns if c != 'target']

numeric_transformer = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
    ]
)

preprocess = ColumnTransformer(
    transformers=[('num', numeric_transformer, feature_cols)],
    remainder='drop'
)

X_processed = preprocess.fit_transform(df_clean[feature_cols])
processed_df = pd.DataFrame(
    X_processed, columns=[f'feature_{i+1}' for i in range(X_processed.shape[1])]
)
processed_df['target'] = df_clean['target'].values

print('Processed shape:', processed_df.shape)
processed_df.head()


In [ ]:
output_path = PREP_DIR / 'breast_cancer_preprocessed.csv'
processed_df.to_csv(output_path, index=False)
print('Dataset siap latih disimpan di:', output_path)
processed_df.describe().T


Dataset siap digunakan untuk tahap modelling selanjutnya. Gunakan file reast_cancer_preprocessed.csv agar pipeline modelling/CI konsisten dengan eksperimen.